## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Library

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

import time

import random
random.seed(777)

from PIL import Image
from torch.autograd import Variable

## GPU Setting

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

print(f'Using PyTorch version: {torch.__version__} / Device: {device}')

Using PyTorch version: 1.8.1+cu101 / Device: cuda


## Hyperparameters

In [4]:
TESTPATH = 'drive/MyDrive/Capstone-Design/dataset/PlantVillage/test' 
REALPATH = 'drive/MyDrive/Capstone-Design/dataset/PlantVillage/real' 

GOOGLENETPATH = 'drive/MyDrive/Capstone-Design/model/PlantVillage/GoogLeNet/model_epoch_100_acc_98.pth'
RESNETPATH = 'drive/MyDrive/Capstone-Design/model/PlantVillage/ResNet/model_epoch_100_acc_98.pth'

BATCHSIZE = 32

## Data Load

In [27]:
transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(), 
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

test_data = torchvision.datasets.ImageFolder(root=TESTPATH, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCHSIZE, shuffle=False, num_workers=2)

real_data = torchvision.datasets.ImageFolder(root=REALPATH, transform=transform_test)
real_loader = torch.utils.data.DataLoader(real_data, batch_size=BATCHSIZE, shuffle=False, num_workers=2)

classes = ('Bacterial_spot', 'Early_blight', 'Healthy', 'Late_blight', 'Leaf_Mold', 
           'Septoria_leaf_spot', 'Spider_mites Two-spotted_spider_mite',
           'Target_Spot', 'Tomato_mosaic_virus', 'Tomato_Yellow_Leaf_Curl_Virus')

## Define Model

In [9]:
class ConvBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.act(x)
        return x

class InceptionModule(nn.Module):
    
    def __init__(self, in_channels, f_1x1, f_3x3_r, f_3x3, f_5x5_r, f_5x5, f_pp):
        super(InceptionModule, self).__init__()
        
        self.branch1 = nn.Sequential(
            ConvBlock(in_channels, f_1x1, kernel_size=1, stride=1, padding=0)
        )
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, f_3x3_r, kernel_size=1, stride=1, padding=0),
            ConvBlock(f_3x3_r, f_3x3, kernel_size=3, stride=1, padding=1)
        )
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, f_5x5_r, kernel_size=1, stride=1, padding=0),
            ConvBlock(f_5x5_r, f_5x5, kernel_size=5, stride=1, padding=2)
        )
        
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1, ceil_mode=True),
            ConvBlock(in_channels, f_pp, kernel_size=1, stride=1, padding=0)
        )
        
    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        
        return torch.cat([branch1, branch2, branch3, branch4], 1)


class InceptionAux(nn.Module):
    
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        
        self.pool = nn.AdaptiveAvgPool2d((4,4))
        self.conv = nn.Conv2d(in_channels, 128, kernel_size=1, stride=1, padding=0)
        self.act = nn.ReLU()
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.pool(x)
        
        x = self.conv(x)
        x = self.act(x)
    
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        
        return x


class GoogLeNet(nn.Module):
    
    def __init__(self, num_classes = 10):
        super(GoogLeNet, self).__init__()
      
        self.conv1 = ConvBlock(3, 64, kernel_size=7, stride=2, padding=3)
        self.pool1 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.conv2 = ConvBlock(64, 64, kernel_size=1, stride=1, padding=0)
        self.conv3 = ConvBlock(64, 192, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.inception3A = InceptionModule(in_channels=192,
                                           f_1x1=64,
                                           f_3x3_r=96,
                                           f_3x3=128,
                                           f_5x5_r=16,
                                           f_5x5=32,
                                           f_pp=32)
        self.inception3B = InceptionModule(in_channels=256,
                                           f_1x1=128,
                                           f_3x3_r=128,
                                           f_3x3=192,
                                           f_5x5_r=32,
                                           f_5x5=96,
                                           f_pp=64)
        self.pool4 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.inception4A = InceptionModule(in_channels=480,
                                           f_1x1=192,
                                           f_3x3_r=96,
                                           f_3x3=208,
                                           f_5x5_r=16,
                                           f_5x5=48,
                                           f_pp=64)
        self.inception4B = InceptionModule(in_channels=512,
                                           f_1x1=160,
                                           f_3x3_r=112,
                                           f_3x3=224,
                                           f_5x5_r=24,
                                           f_5x5=64,
                                           f_pp=64)
        self.inception4C = InceptionModule(in_channels=512,
                                           f_1x1=128,
                                           f_3x3_r=128,
                                           f_3x3=256,
                                           f_5x5_r=24,
                                           f_5x5=64,
                                           f_pp=64)
        self.inception4D = InceptionModule(in_channels=512,
                                           f_1x1=112,
                                           f_3x3_r=144,
                                           f_3x3=288,
                                           f_5x5_r=32,
                                           f_5x5=64,
                                           f_pp=64)
        self.inception4E = InceptionModule(in_channels=528,
                                           f_1x1=256,
                                           f_3x3_r=160,
                                           f_3x3=320,
                                           f_5x5_r=32,
                                           f_5x5=128,
                                           f_pp=128)
        self.pool5 = nn.MaxPool2d(3, stride=2, padding=0, ceil_mode=True)
        self.inception5A = InceptionModule(in_channels=832,
                                           f_1x1=256,
                                           f_3x3_r=160,
                                           f_3x3=320,
                                           f_5x5_r=32,
                                           f_5x5=128,
                                           f_pp=128)
        self.inception5B = InceptionModule(in_channels=832,
                                           f_1x1=384,
                                           f_3x3_r=192,
                                           f_3x3=384,
                                           f_5x5_r=48,
                                           f_5x5=128,
                                           f_pp=128)
        self.pool6 = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)
        
        self.aux4A = InceptionAux(512, num_classes) 
        self.aux4D = InceptionAux(528, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.inception3A(x)
        x = self.inception3B(x)
        x = self.pool4(x)
        x = self.inception4A(x)
  
        aux1 = self.aux4A(x)
        
        x = self.inception4B(x)
        x = self.inception4C(x)
        x = self.inception4D(x)
  
        aux2 = self.aux4D(x)
        
        x = self.inception4E(x)
        x = self.pool5(x)
        x = self.inception5A(x)
        x = self.inception5B(x)
        x = self.pool6(x)
        x = torch.flatten(x,1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x, aux1, aux2


googlenet = GoogLeNet().to(device)

In [10]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes) #conv1x1(64,64)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride) #conv3x3(64,64)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion) #conv1x1(64,256)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x) # 1x1 stride = 1
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out) # 3x3 stride = stride 
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out) # 1x1 stride = 1
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()

        self.inplanes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

resnet = ResNet(Bottleneck, [3, 4, 6, 3], 10, True).to(device)

## Load Model

In [14]:
googlenet.load_state_dict(torch.load(GOOGLENETPATH))
resnet.load_state_dict(torch.load(RESNETPATH))

<All keys matched successfully>

## Performance

In [15]:
IMAGEPATH = 'drive/MyDrive/Capstone-Design/dataset/PlantVillage/test/0/0_0.jpg'

trans = transforms.Compose([
                             transforms.Resize(224),
                             transforms.ToTensor(),
                             transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

def image_loader(image_name):
  image = Image.open(image_name)
  image = trans(image).float()
  image = Variable(image, requires_grad=True)
  image = image.unsqueeze(0)
  return image.cuda()

image = image_loader(IMAGEPATH)

In [16]:
def model_accuracy(valid_loader, model, google=False):
  correct = 0
  total = 0

  class_correct = list(0. for i in range(7))
  class_total = list(0. for i in range(7))

  with torch.no_grad():
      for data in valid_loader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)

          if google:
            outputs, aux1, aux2 = model(images)
          else:
            outputs = model(images)

          _, predicted = torch.max(outputs, 1)

          total += labels.size(0)
          correct += (predicted == labels).sum().item()

          c = (predicted == labels).squeeze()
          for i in range(len(labels)):
              label = labels[i]
              class_correct[label] += c[i].item()
              class_total[label] += 1

  print(f'Accuracy of the network on the {len(valid_data)} test images: {(100 * correct)/total:.2f} %')

  for i in range(7):
      print('Accuracy of %s : %.2f %%' % (
          classes[i], 100 * class_correct[i] / class_total[i]))

In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def performance(valid_loader, model, google_net=False):
  test = []
  pred = []

  model.eval()

  with torch.no_grad():
    for data in valid_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      
      if google_net:
        outputs, aux1, aux2 = model(images)
      else:
        outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)

      for i in range(len(labels)):
        test.append(int(labels[i]))
        pred.append(int(predicted[i]))

  confusion = confusion_matrix(test, pred)
  accuracy = accuracy_score(test, pred)
  precision = precision_score(test, pred, average='macro')
  recall = recall_score(test, pred, average='macro')
  f1 = f1_score(test, pred, average='macro')

  print('Confusion Matrix')
  print(confusion)
  print()
  print(f'accuracy  : {accuracy:.6}')
  print(f'precision : {precision:.6}')
  print(f'recall    : {recall:.6}')
  print(f'f1-score  : {f1:.6}')
    
  

In [18]:
def single_predict(image, model, google_net=False):
  start = time.time()
  if google_net:
    outputs, aux1, aux2 = model(image)
  else:
    outputs = model(image)
  _, predicted = torch.max(outputs.data, 1)
  end = time.time()
  print(f'predicted {predicted}')
  print(f'time {end - start}')

In [24]:
def performances(valid_loader, image, model, google=False):
  performance(valid_loader, model, google)
  print()
  # model_accuracy(valid_loader, model, google)
  # print()
  # single_predict(image, model, google)

In [28]:
print('--------------GoogLeNet--------------')

print('Test Data')
performances(test_loader, image, googlenet, True)
print()

print('Real Data')
performances(real_loader, image, googlenet, True)

--------------GoogLeNet--------------
Test Data
Confusion Matrix
[[ 421    0    1    0    0    0    0    1    0    2]
 [   0  193    0    6    0    0    0    1    0    0]
 [   0    0  318    0    0    0    0    0    0    0]
 [   0    2    0  379    0    0    0    0    0    0]
 [   0    0    0    0  187    0    2    0    0    1]
 [   0    0    0    1    0  353    0    0    0    0]
 [   0    0    0    0    0    0  333    2    0    0]
 [   0    1    0    1    0    1    6  271    0    0]
 [   0    0    0    0    0    0    0    0   74    0]
 [   0    0    0    0    0    0    0    1    0 1070]]

accuracy  : 0.992007
precision : 0.991369
recall    : 0.989268
f1-score  : 0.990281


Real Data
Confusion Matrix
[[0 0 0 7 0 3 0 0 0 0]
 [0 9 0 1 0 0 0 0 0 0]
 [0 0 0 9 0 1 0 0 0 0]
 [0 2 0 7 0 1 0 0 0 0]
 [0 1 0 8 0 0 0 0 0 1]
 [0 7 0 3 0 0 0 0 0 0]
 [0 0 1 9 0 0 0 0 0 0]
 [0 2 0 5 0 2 0 0 0 1]
 [0 2 0 8 0 0 0 0 0 0]
 [0 1 0 9 0 0 0 0 0 0]]

accuracy  : 0.16
precision : 0.0481061
recall    : 0.16
f1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
print('--------------ResNet--------------')

print('Test Data')
performances(test_loader, image, resnet, False)
print()

print('Real Data')
performances(real_loader, image, resnet, False)

--------------ResNet--------------
Test Data
Confusion Matrix
[[ 421    0    0    1    0    0    0    1    0    2]
 [   2  196    0    0    0    0    0    1    0    1]
 [   0    0  318    0    0    0    0    0    0    0]
 [   2    5    0  370    0    2    1    1    0    0]
 [   0    0    1    1  188    0    0    0    0    0]
 [   0    0    0    1    0  352    0    1    0    0]
 [   0    1    1    0    0    0  332    0    1    0]
 [   1    1    0    0    1    2    6  269    0    0]
 [   0    0    0    0    0    0    1    0   73    0]
 [   1    0    0    0    1    0    0    0    0 1069]]

accuracy  : 0.988975
precision : 0.986092
recall    : 0.986192
f1-score  : 0.986102


Real Data
Confusion Matrix
[[0 4 0 6 0 0 0 0 0 0]
 [0 7 0 3 0 0 0 0 0 0]
 [0 0 0 9 0 0 0 0 0 1]
 [0 2 0 8 0 0 0 0 0 0]
 [0 1 0 9 0 0 0 0 0 0]
 [0 4 0 4 0 2 0 0 0 0]
 [0 0 1 9 0 0 0 0 0 0]
 [0 3 0 5 0 1 0 0 0 1]
 [0 3 0 6 0 0 0 0 0 1]
 [0 5 0 5 0 0 0 0 0 0]]

accuracy  : 0.17
precision : 0.103305
recall    : 0.17
f1-sco

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
